In [1]:
## 네이버 영화감상

# import urllib.request

# urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt",
#                             filename="./data/ratings_train.txt")
# urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt",
#                             filename="./data/ratings_test.txt")

In [2]:
## 저장된 ./data/ratings_train.txt 로 영화 리뷰 모델링을 학습해서 모델 생성
## ./data/ratings_train.txt 로 정확도를 확인해서 출력

# 1. 데이터 로드
# 2. 토큰화
# 3. 단어 -> 숫자로 변화 시킴
# 4. 문장의 사이즈를 동일하게
# 5. 모델 설계 후 실행

In [3]:
# %load tensor.py
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [4]:
data_train = pd.read_table('./data/ratings_train.txt')
data_test =  pd.read_table('./data/ratings_test.txt')

In [5]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [6]:
data_train['document'].value_counts()

document
굿                                                                                                  181
good                                                                                                92
최고                                                                                                  85
쓰레기                                                                                                 79
별로                                                                                                  66
                                                                                                  ... 
이런거 만드는 색휘는 진짜 일부로 욕처묵고싶어서 환장한색휘인듯                                                                   1
이영화 평점이 궁금해서 왔는데.너무높다..^^                                                                            1
시청자들에게 사과하세여 제작진..낚시성 예고 지친다..보기싫음 말아라 하고 만듬?조세호씨 때문에 넘웃기게 보는데요 낚시그만하시고 예고와 본방이 이어지는 연출부탁드립니다..      1
여태본 영화중 단연 최고                                                   

In [7]:
data_train.drop_duplicates(subset=['document'], inplace = True)

In [8]:
data_train['document'].value_counts()

document
아 더빙.. 진짜 짜증나네요 목소리                                                                            1
여러분은 지금 톰하디의 BMW광고를 보고계십니다                                                                     1
무슨 성교육용 비디오도 아니고 이건 대체..                                                                       1
여 형사 죽일때 부터 직쏘의 코드에서 벗어나 있었다 아만다로 인해 쏘우를 버리고 단순한 삼류 고어물로 바뀜                                    1
재미없어요..진심입니다                                                                                   1
                                                                                              ..
이게 왜 수작이라는지 도저히 이해가 안되는 ...                                                                    1
이 영화는 강우석 박찬욱영화보다 어떤의미에서는 정말 뛰어나다고 생각하는 1인. 한국 최고의 영화감독 윤성호.정말 주인공이 윤성호감독과 말하는스타일이 너무 비슷해서.    1
쎄다.                                                                                            1
ㅋㅋ진짜 재밌게 봤음 ㅋ                                                                                  1
한국 영화 최초로 수간하는 내용이 담긴

In [9]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 146183 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        146183 non-null  int64 
 1   document  146182 non-null  object
 2   label     146183 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.5+ MB


In [10]:
# 한글을 제외한 문자를 '' 로 변경
data_train['document'] = data_train['document'].str.replace(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 " "]', 
                                                          '', regex=True)
# '' 를 np.nan  으로 변경 -> nan 을 제거
data_train['document'].replace('', np.nan, inplace=True)
data_train.dropna(inplace=True)

print(len(data_train))

145795


In [11]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 145795 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        145795 non-null  int64 
 1   document  145795 non-null  object
 2   label     145795 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.4+ MB


In [12]:
x_train = data_train['document']
y_train = data_train['label']

In [13]:
# 1. 토큰화

token = Tokenizer()
token.fit_on_texts(x_train) # 리뷰 단어 토큰화

# print("단어의 인덱스 : ", token.word_index,'\n', len(token.word_index)) #총 단어수

In [14]:
# 토큰한 결과를 가지고 인덱스로 문장을 변환
x = token.texts_to_sequences(x_train) # 문장의 단어를 token한 결과 단어의 인덱스로 변환

# print(x)

In [15]:
padd_x = pad_sequences(x, 8)

In [18]:
# 딥러닝 모델 생성
model = Sequential()
model.add(Embedding(len(token.word_index) + 1, 8, input_length = 8)) # 총 단어수, 임베딩 사이즈, 입력 단어수
model.add(Flatten()) # 다음 레이어를 위해 1차원 배열로 변환
model.add(Dense(1,activation = 'sigmoid')) # 긍정 또는 부정이므로 활성함수는 sigmoid
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 8, 8)              2396144   
                                                                 
 flatten (Flatten)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 2,396,209
Trainable params: 2,396,209
Non-trainable params: 0
_________________________________________________________________


In [19]:
# 모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# 모델 실행
model.fit(padd_x, y_train, epochs= 20)
print('Accuracy : ', model.evaluate(padd_x, y_train)[1])

Epoch 1/20
4557/4557 [==============================] - 105s 23ms/step - loss: 0.4992 - accuracy: 0.7500
Epoch 2/20
4557/4557 [==============================] - 104s 23ms/step - loss: 0.2589 - accuracy: 0.9128
Epoch 3/20
4557/4557 [==============================] - 104s 23ms/step - loss: 0.1286 - accuracy: 0.9595
Epoch 4/20
4557/4557 [==============================] - 103s 23ms/step - loss: 0.0703 - accuracy: 0.9770
Epoch 5/20
4557/4557 [==============================] - 103s 23ms/step - loss: 0.0439 - accuracy: 0.9856
Epoch 6/20
4557/4557 [==============================] - 105s 23ms/step - loss: 0.0310 - accuracy: 0.9897
Epoch 7/20
4557/4557 [==============================] - 104s 23ms/step - loss: 0.0229 - accuracy: 0.9923
Epoch 8/20
4557/4557 [==============================] - 104s 23ms/step - loss: 0.0185 - accuracy: 0.9936
Epoch 9/20
4557/4557 [==============================] - 103s 23ms/step - loss: 0.0153 - accuracy: 0.9946
Epoch 10/20
4557/4557 [==============================] 

In [21]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        50000 non-null  int64 
 1   document  49997 non-null  object
 2   label     50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


In [22]:
data_test.drop_duplicates(subset=['document'], inplace = True)

In [23]:
data_test['document'].value_counts()

document
굳 ㅋ                                                                                                                                            1
4대강 살리긴는 절대로 안돼지라잉. 우덜 영산강만 살리면 돼제. 4대강 살리기 운운하는 넘있으면 배때지 쑤셔불고 민주화 시켜부러야돼. 카악퉤~~~                                                              1
영화라고 부르기도 힘든 졸작                                                                                                                                1
요즘엔 운널사 덕분에 행복한한주를 보내여!!..장혁씨의 연기력에 늘 박수를 보내며 스토리 진짜 짱!!!!..장나라씨도 넘 이쁘고 착하구 연기도 잘하시구,,조주연분들역시짱..나쁜 악역이 판치지않아 보는내내 힐링+해피랍니다...ㅎㅎ이런좋은잘품볼수있어쌩유    1
웃음도 안나오는 개판.                                                                                                                                   1
                                                                                                                                              ..
정말 재미있었습니다. 앞으로도 좋은 이야기 부탁드려요~^^ 프로포즈대작전 파이팅!                                                                            

In [24]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 49158 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        49158 non-null  int64 
 1   document  49157 non-null  object
 2   label     49158 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.5+ MB


In [25]:
# 한글을 제외한 문자를 '' 로 변경
data_test['document'] = data_test['document'].str.replace(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 " "]', 
                                                          '', regex=True)
# '' 를 np.nan  으로 변경 -> nan 을 제거
data_test['document'].replace('', np.nan, inplace=True)
data_test.dropna(inplace=True)

print(len(data_test))

48995


In [26]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 48995 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        48995 non-null  int64 
 1   document  48995 non-null  object
 2   label     48995 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.5+ MB


In [27]:
x_test = data_test['document']
y_test = data_test['label']

In [28]:
# 1. 토큰화

token = Tokenizer()
token.fit_on_texts(x_test) # 리뷰 단어 토큰화

In [29]:
# 토큰한 결과를 가지고 인덱스로 문장을 변환
x_test_1 = token.texts_to_sequences(x_test) # 문장의 단어를 token한 결과 단어의 인덱스로 변환

In [30]:
padd_x_test = pad_sequences(x_test_1, 8)

In [31]:
model.evaluate(padd_x_test, y_test)

1532/1532 [==============================] - 2s 1ms/step - loss: 3.7303 - accuracy: 0.5041


[3.7303245067596436, 0.5041330456733704]